In [ ]:
import pandas as pd
from tqdm import tqdm
import time
import requests
from bs4 import BeautifulSoup
from datetime import datetime
def date_range(start, end):
    start = datetime.strptime(start, "%Y%m%d")
    end = datetime.strptime(end, "%Y%m%d")
    dates = [date.strftime("%Y%m%d") for date in pd.date_range(start, periods=(end-start).days+1)]
    return dates
    


In [ ]:
def 공시찾기(start, end, num ):
    '''
    start(str):검색시작할 날짜
    end(str):검색마지막 날짜
    num(int):api키 순서 0~4까지 선택가능
    pblntf: 공시상세구분
    '''
    공시분류={
        'A001':'사업보고서',
        'A002':'반기보고서',
        'A003':'분기보고서',
        'A004':'등록법인결산서류(자본시장법이전)',
        'A005':'소액공모법인결산서류',
        'B001':'주요사항보고서',
        'B002':'주요경영사항신고(자본시장법 이전)',
        'B003':'최대주주등과의거래신고(자본시장법 이전)',
        'C001':'증권신고(지분증권)',
        'C002':'증권신고(채무증권)',
        'C003':'증권신고(파생결합증권)',
        'C004':'증권신고(합병등)',
        'C005':'증권신고(기타)',
        'C006':'소액공모(지분증권)',
        'C007':'소액공모(채무증권)',
        'C008':'소액공모(파생결합증권)',
        'C009':'소액공모(합병등)',
        'C010':'소액공모(기타)',
        'C011':'호가중개시스템을통한소액매출',
        'D001':'주식등의대량보유상황보고서',
        'D002':'임원ㆍ주요주주특정증권등소유상황보고서',
        'D003':'의결권대리행사권유',
        'D004':'공개매수',
        'E001':'자기주식취득/처분',
        'E002':'신탁계약체결/해지',
        'E003':'합병등종료보고서',
        'E004':'주식매수선택권부여에관한신고',
        'E005':'사외이사에관한신고',
        'E006':'주주총회소집공고',
        'E007':'시장조성/안정조작',
        'E008':'합병등신고서(자본시장법 이전)',
        'E009':'금융위등록/취소(자본시장법 이전)',
        'F001':'감사보고서',
        'F002':'연결감사보고서',
        'F003':'결합감사보고서',
        'F004':'회계법인사업보고서',
        'F005':'감사전재무제표미제출신고서',
        'G001':'증권신고(집합투자증권-신탁형)',
        'G002':'증권신고(집합투자증권-회사형)',
        'G003':'증권신고(집합투자증권-합병)',
        'H001':'자산유동화계획/양도등록',
        'H002':'사업/반기/분기보고서',
        'H003':'증권신고(유동화증권등)',
        'H004':'채권유동화계획/양도등록',
        'H005':'수시보고',
        'H006':'주요사항보고서',
        'I001':'수시공시',
        'I002':'공정공시',
        'I003':'시장조치/안내',
        'I004':'지분공시',
        'I005':'증권투자회사',
        'I006':'채권공시',
        'J001':'대규모내부거래관련',
        'J002':'대규모내부거래관련(구)',
        'J004':'기업집단현황공시',
        'J005':'비상장회사중요사항공시',
        'J006':'기타공정위공시',
        'J008':'대규모내부거래관련(공익법인용)'
        }

    key=['74c96b049945efc363e3400482f03c6dda33a71a',
           '652c42586af3946cf42600d79968d6448e805a17',
           '68fc9f25c238876fa53759e6a3a5b653110020bd',
           'd7cb577248187e41f1c62e0630918e9dec47087c',
           '68fc9f25c238876fa53759e6a3a5b653110020bd']
    dates = date_range(start, end)
    q1=dates[:80]
    q2=dates[80:160]
    q3=dates[160:240]
    q4=dates[240:320]
    q5=dates[320:]
    quater=[q1,q2,q3,q4,q5]
    df_공시=pd.DataFrame({'corp_code':[0],'corp_name':[0],'stock_code':[0],'corp_cls':[0],'report_nm':[0],'rcept_no':[0],'flr_nm':[0],'rcept_dt':[0],'rm':[0],'공시분류':[0]})
    cnt=0
    temp_before=1
    for date in tqdm(quater):

        temp=pd.DataFrame({'corp_code':[1],'corp_name':[1],'stock_code':[1],'corp_cls':[1],'report_nm':[1],'rcept_no':[1],'flr_nm':[1],'rcept_dt':[1],'rm':[1],'공시분류':[1]})
        #print(date)
        for i,k in 공시분류.items():
            page_no=0
            try:
                while True: 
                    page_no+=1
                    cnt+=1
                    page_count=100
                    #print()
                    crtfc_key=key[num]# api 키 리스트
                    url=f'https://opendart.fss.or.kr/api/list.json?crtfc_key={crtfc_key}&bgn_de={date[0]}&end_de={date[-1]}&page_no={page_no}&page_count={page_count}&pblntf_detail_ty={i}'
                    temp_before=temp.iloc[-1,5]
                    temp=pd.DataFrame(requests.get(url).json()['list'])
                    temp['공시분류']=k

                    df_공시=df_공시.append(temp)
                    #time.sleep(0.2)
                    #print(page_no)
                    if temp_before ==temp.iloc[-1,5]:
                        break
                    else:
                        pass


            except:
                pass
    print(cnt)
    df_공시=df_공시.drop_duplicates()
    return df_공시

In [ ]:
df_2020=공시찾기('20200101', '20201231', 1)
df_2020=df_2020[1:]

100%|██████████| 5/5 [28:10<00:00, 338.10s/it]


2565


In [ ]:
df_2021=공시찾기('20210101', '20211231', 2)
df_2021=df_2021[1:]

100%|██████████| 5/5 [31:17<00:00, 375.44s/it]


2572


In [ ]:
df_2019=공시찾기('20190101', '20191231', 2)
df_2019=df_2019[1:]

100%|██████████| 5/5 [28:00<00:00, 336.18s/it]


2503


In [ ]:
df_2018=공시찾기('20180101', '20181231', 3)
df_2018=df_2018[1:]

100%|██████████| 5/5 [26:25<00:00, 317.15s/it]


2437


In [ ]:
df_2017=공시찾기('20170101', '20171231', 3)
df_2017=df_2017[1:]

100%|██████████| 5/5 [23:37<00:00, 283.40s/it]


2306


In [ ]:
df_공시=pd.concat([df_2017,df_2018,df_2019,df_2020,df_2021])

In [ ]:
df_공시.to_csv('공시정보.csv')

In [ ]:
df_공시.groupby('공시분류').count()

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
공시분류,,,,,,,,,
감사보고서,151458,151458,151458,151458,151458,151458,151458,151458,151458
감사전재무제표미제출신고서,14,14,14,14,14,14,14,14,14
공개매수,303,303,303,303,303,303,303,303,303
공정공시,12069,12069,12069,12069,12069,12069,12069,12069,12069
기업집단현황공시,43359,43359,43359,43359,43359,43359,43359,43359,43359
대규모내부거래관련,33758,33758,33758,33758,33758,33758,33758,33758,33758
대규모내부거래관련(구),1,1,1,1,1,1,1,1,1
반기보고서,14117,14117,14117,14117,14117,14117,14117,14117,14117
분기보고서,28141,28141,28141,28141,28141,28141,28141,28141,28141


In [ ]:
import joblib

joblib.dump(df_공시,'df_공시.pkl')